Import

In [2]:
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data = pd.read_csv("./Dataset/all_catagories/Processed_all_datas.csv")
print(data.shape)

(8600, 13)


In [4]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date
0,US,51632887,R3B581VNKYCP16,B00HFPOXM4,114966677,Garmin Vivofit Fitness Band,Wireless,5.0,5854.0,6063.0,N,Y,2014-03-12
1,US,49452274,RT0KPKVSQD0HI,B00A17IAO0,979081054,UP by Jawbone Wristband,Wireless,5.0,4856.0,5029.0,N,Y,2012-11-27
2,US,44086587,RYBUAAD9JZ1VW,B001S2RCWI,835787812,Garmin Portable Friction Dashboard Mount,Wireless,5.0,4017.0,4048.0,N,Y,2009-06-14
3,US,49452274,R2Z0F95XGL71C6,B00GOGV314,190508754,"UP24 by Jawbone Wristband, Retail Packaging",Wireless,5.0,3914.0,4022.0,N,N,2013-12-08
4,US,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,212863722,"Wemo Wi-Fi enabled, Works with Amazon Alexa",Wireless,1.0,3667.0,3894.0,N,Y,2013-11-01


In [5]:
data.dropna(subset=['marketplace','product_parent','product_title','product_category'],inplace=True,axis=0)
data=data.drop_duplicates(subset=['product_id'])
print(data.shape)

(5946, 13)


In [6]:
#remove Nan
data.dropna(subset=['marketplace','product_parent','product_title','product_category'],inplace=True,axis=0)
#Reset index
data = data.reset_index(drop=True)

In [7]:
#Remove non-alphanumeric characters from columns:
data['marketplace'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['marketplace']]
data['product_parent'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_parent']]
data['product_title'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_title']]
data['product_category'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_category']]

In [8]:
data["combined"] = data['marketplace'] + '  ' + data['product_parent'] + ' ' + data['product_title'] + ' ' + data['product_category']
data.drop(['marketplace','product_parent','product_category'],axis=1,inplace=True)
data.head()

,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,combined
0,51632887,R3B581VNKYCP16,B00HFPOXM4,Garmin Vivofit Fitness Band,5.0,5854.0,6063.0,N,Y,2014-03-12,US 114966677 Garmin Vivofit Fitness Band Wire...
1,49452274,RT0KPKVSQD0HI,B00A17IAO0,UP by Jawbone Wristband,5.0,4856.0,5029.0,N,Y,2012-11-27,US 979081054 UP by Jawbone Wristband Wireless
2,44086587,RYBUAAD9JZ1VW,B001S2RCWI,Garmin Portable Friction Dashboard Mount,5.0,4017.0,4048.0,N,Y,2009-06-14,US 835787812 Garmin Portable Friction Dashboa...
3,49452274,R2Z0F95XGL71C6,B00GOGV314,UP24 by Jawbone Wristband Retail Packaging,5.0,3914.0,4022.0,N,N,2013-12-08,US 190508754 UP24 by Jawbone Wristband Retail...
4,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,Wemo WiFi enabled Works with Amazon Alexa,1.0,3667.0,3894.0,N,Y,2013-11-01,US 212863722 Wemo WiFi enabled Works with Ama...


In [9]:
data

,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,combined
0,51632887,R3B581VNKYCP16,B00HFPOXM4,Garmin Vivofit Fitness Band,5.0,5854.0,6063.0,N,Y,2014-03-12,US 114966677 Garmin Vivofit Fitness Band Wire...
1,49452274,RT0KPKVSQD0HI,B00A17IAO0,UP by Jawbone Wristband,5.0,4856.0,5029.0,N,Y,2012-11-27,US 979081054 UP by Jawbone Wristband Wireless
2,44086587,RYBUAAD9JZ1VW,B001S2RCWI,Garmin Portable Friction Dashboard Mount,5.0,4017.0,4048.0,N,Y,2009-06-14,US 835787812 Garmin Portable Friction Dashboa...
3,49452274,R2Z0F95XGL71C6,B00GOGV314,UP24 by Jawbone Wristband Retail Packaging,5.0,3914.0,4022.0,N,N,2013-12-08,US 190508754 UP24 by Jawbone Wristband Retail...
4,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,Wemo WiFi enabled Works with Amazon Alexa,1.0,3667.0,3894.0,N,Y,2013-11-01,US 212863722 Wemo WiFi enabled Works with Ama...
...,...,...,...,...,...,...,...,...,...,...,...
5941,50119287,RG13K28T0JHD,0060763280,Secrets of the Millionaire Mind Mastering the ...,5.0,941.0,990.0,N,N,2005-03-23,US 846347314 Secrets of the Millionaire Mind ...
5942,52827802,R3SWJQ4BEIEHDF,0385509510,Under the Banner of Heaven A Story of Violent ...,5.0,873.0,988.0,N,N,2003-09-26,US 418900469 Under the Banner of Heaven A Sto...
5943,48890006,RPP8P15M3U2JL,0879838167,Nutrition and Physical Degeneration,5.0,961.0,980.0,N,Y,2004-01-21,US 472643259 Nutrition and Physical Degenerat...
5944,50823784,R2MXU0D7H4MPX8,0465016901,The Drama of the Gifted Child The Search for t...,4.0,950.0,980.0,N,N,2000-11-07,US 241968441 The Drama of the Gifted Child Th...


In [10]:
# Content Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['combined'])

#Finding cosine similarity between vectors
cosine_similarities = cosine_similarity(tfidf_matrix,tfidf_matrix)
# Tạo một Series với chỉ mục là tiêu đề sản phẩm và giá trị là chỉ mục tương ứng trong dữ liệu
p_title = data['product_title']
indices = pd.Series(data.index, index=data['product_title'])

In [11]:
tfidf_matrix.get_shape()

(5946, 16982)

In [12]:
name = vectorizer.get_feature_names_out()
df_tf_idf = pd.DataFrame(tfidf_matrix .toarray(), columns = name)
v = df_tf_idf.iloc[[80]]
v = v.loc[:, v.any()]
print(v)

    578435577    galaxy        s5   samsung       us  wireless
80   0.586151  0.422566  0.511203  0.366558  0.06822  0.278419
